In [ ]:
from epx import Job, ModelConfig, SynthPop

from pathlib import Path
import os

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

os.environ["FRED_PROJECT"] = str(Path.cwd())
path_to_fred_project = os.environ["FRED_PROJECT"]

In [ ]:
mpox_config = ModelConfig(synth_pop=SynthPop("US_2010.v5", ["Dane_County_WI"]),
    start_date = "2022-06-01",
    end_date = "2024-06-01")

results_dir = "/home/epx/cl-results"

# Configure FRED job
mpox_job = Job(
    "model/main_monkeypox.fred",
    config=[mpox_config],
    key="mpox_job",
    results_dir=results_dir,
    size="hot",
    # Select FRED version compatible with selected model
    fred_version="11.0.1"
)

# Execute job
mpox_job.execute()

import time
# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(mpox_job.status) != 'DONE':
    if str(mpox_job.status) == 'ERROR':
        logs = mpox_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(mpox_job.status)

In [ ]:
mpox_job.results.state(condition="BATHHOUSE", state="Visit", count_type="new")

In [ ]:
susceptible = mpox_job.results.state(
    condition="MPOX",
    state="Susceptible",
    count_type="new"
)

exposed = mpox_job.results.state(
    condition="MPOX",
    state="Exposed",
    count_type="new"
)

infectious = mpox_job.results.state(
    condition="MPOX",
    state="Infectious",
    count_type="new"
)


recovered = mpox_job.results.state(
    condition="MPOX",
    state="Recovered",
    count_type="new"
)

In [ ]:
susceptible

In [ ]:
susceptible.to_csv("sus_counts.csv")

In [ ]:
# rename the 'new' counts column to the name of the state
susceptible.rename(columns={'new':'Susceptible'},inplace=True)
exposed.rename(columns={'new':'Exposed'},inplace=True)
infectious.rename(columns={'new':'Infectious'},inplace=True)
recovered.rename(columns={'new':'Recovered'},inplace=True)

# build up the merged dataframe with counts for all states
all_states = pd.merge(susceptible,exposed)
all_states = pd.merge(all_states,infectious)
all_states = pd.merge(all_states,recovered)

#retrieve the sim_date from the column and add to the all_states frame.
all_states['sim_date']=mpox_job.results.dates().sim_date

In [ ]:
all_states.to_csv("all_mpox_data_vals1.csv")

In [ ]:
all_states['Exposed_7day_avg'] = all_states.Exposed.rolling(7).mean()
all_states['Recovered_7day_avg'] = all_states.Recovered.rolling(7).mean()
all_states['Infectious_7day_avg'] = all_states.Infectious.rolling(7).mean()

all_state_7day_df = all_states[~all_states['Exposed_7day_avg'].isnull()]

fig = px.line(
    all_state_7day_df,
    x='sim_date',
    y=['Exposed_7day_avg','Infectious_7day_avg','Recovered_7day_avg'], # pass a list to plot all states],
    title="Mpox outbreak in County")
fig.update_layout( # customize font and legend orientation & position
    font_family="Epistemix Label",
    yaxis_title="New infections per day, 7 day average",
    xaxis_title="Date",
    legend_title="State",
    title_font_size=30
)
fig.show()

In [ ]:
mpox_job.delete(interactive=False)